# Data Exploration

Let's load a dataset with the pyspark

## Packages

In [1]:
from pyspark.sql import SQLContext
from pyspark import SparkConf,SparkContext

In [2]:
conf =  SparkConf().setMaster("local").setAppName("daily_water")
sc = SparkContext(conf =conf)
sqlContext = SQLContext(sc)

In [5]:
# the second argument specifies the spark-csv format, the last one says to infer the data types
df = sqlContext.read.load("../datasets/daily_weather.csv", 
                          format="com.databricks.spark.csv",
                          header=True, inferSchema="true")

In [4]:
type(df)

pyspark.sql.dataframe.DataFrame

In [5]:
df.columns

['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [6]:
df.dtypes

[('number', 'int'),
 ('air_pressure_9am', 'double'),
 ('air_temp_9am', 'double'),
 ('avg_wind_direction_9am', 'double'),
 ('avg_wind_speed_9am', 'double'),
 ('max_wind_direction_9am', 'double'),
 ('max_wind_speed_9am', 'double'),
 ('rain_accumulation_9am', 'double'),
 ('rain_duration_9am', 'double'),
 ('relative_humidity_9am', 'double'),
 ('relative_humidity_3pm', 'double')]

In [7]:
df.printSchema()

root
 |-- number: integer (nullable = true)
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: double (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: double (nullable = true)



In [8]:
df.describe().toPandas().T

0                    1                   2  \
summary                 count                 mean              stddev   
number                   1095                547.0  316.24357700987383   
air_pressure_9am         1092    918.8825513138094   3.184161180386833   
air_temp_9am             1090    64.93300141287072  11.175514003175877   
avg_wind_direction_9am   1091    142.2355107005759   69.13785928889189   
avg_wind_speed_9am       1092     5.50828424225493  4.5528134655317185   
max_wind_direction_9am   1092   148.95351796516923   67.23801294602953   
max_wind_speed_9am       1091    7.019513529175272   5.598209170780958   
rain_accumulation_9am    1089  0.20307895225211126  1.5939521253574893   
rain_duration_9am        1092    294.1080522756142  1598.0787786601481   
relative_humidity_9am    1095    34.24140205923536  25.472066802250055   
relative_humidity_3pm    1095    35.34472714825898  22.524079453587273   

                                         3                   4  
summary                                min                 max  
number                                   0                1094  
air_pressure_9am         907.9900000000024   929.3200000000012  
air_temp_9am            36.752000000000685   98.90599999999992  
avg_wind_direction_9am  15.500000000000046               343.4  
avg_wind_speed_9am        0.69345139999974  23.554978199999763  
max_wind_direction_9am   28.89999999999991  312.19999999999993  
max_wind_speed_9am      1.1855782000000479   29.84077959999996  
rain_accumulation_9am                  0.0   24.01999999999907  
rain_duration_9am                      0.0             17704.0  
relative_humidity_9am    6.090000000001012    92.6200000000002  
relative_humidity_3pm   5.3000000000006855    92.2500000000003

In [9]:
# mean rounded
round(float(df.describe().toPandas().max_wind_speed_9am[4]),2)

29.84

In [10]:
df.describe("air_pressure_9am").show()

+-------+-----------------+
|summary| air_pressure_9am|
+-------+-----------------+
|  count|             1092|
|   mean|918.8825513138094|
| stddev|3.184161180386833|
|    min|907.9900000000024|
|    max|929.3200000000012|
+-------+-----------------+



In [11]:
df.toPandas().shape

(1095, 11)

In [12]:
df.count()

1095

Let's see the number of NaN in our dataset

In [13]:
df.toPandas().isna().sum()

number                    0
air_pressure_9am          3
air_temp_9am              5
avg_wind_direction_9am    4
avg_wind_speed_9am        3
max_wind_direction_9am    3
max_wind_speed_9am        4
rain_accumulation_9am     6
rain_duration_9am         3
relative_humidity_9am     0
relative_humidity_3pm     0
dtype: int64

We are going to erase them 

In [14]:
df2 = df.na.drop(subset="air_pressure_9am")

In [15]:
df2.count()

1092

Let's compute the correlation between two columns 

In [16]:
# In spite of both variables have NaN we can use the method 
df2.stat.corr("rain_accumulation_9am","rain_duration_9am")

0.7298253479609021

In [17]:
round(df.stat.corr("relative_humidity_9am", "relative_humidity_3pm"),2)

0.88

In [18]:
df.toPandas().max_wind_speed_9am.max()

29.84077959999996

In [19]:
#What is the approximate maximum max_wind_direction_9am when the maximum max_wind_speed_9am occurs?
round(df.toPandas().loc[df.toPandas().max_wind_speed_9am == df.toPandas().max_wind_speed_9am.max(),].max_wind_direction_9am.max(), 2)

68.0

In [20]:
"""
If the histogram for air temperature at 9am has 50 bins, 
what is the number of elements 
in the bin with the most elements (without removing or imputing 
missing values)?
"""
df.toPandas().air_temp_9am.describe()

count    1090.000000
mean       64.933001
std        11.175514
min        36.752000
25%        57.281000
50%        65.715479
75%        73.450974
max        98.906000
Name: air_temp_9am, dtype: float64

Let's drop all the missing values

In [24]:
removeAllDF = df.na.drop()

In [26]:
removeAllDF.toPandas().isna().sum()

number                    0
air_pressure_9am          0
air_temp_9am              0
avg_wind_direction_9am    0
avg_wind_speed_9am        0
max_wind_direction_9am    0
max_wind_speed_9am        0
rain_accumulation_9am     0
rain_duration_9am         0
relative_humidity_9am     0
relative_humidity_3pm     0
dtype: int64

Let's impute the NaN with the mean 

In [33]:
from pyspark.sql.functions import avg

inputeDF = df

for x in inputeDF.columns:
    meanValue = removeAllDF.agg(avg(x)).first()[0]
    print(x, meanValue)
    inputeDF = inputeDF.na.fill(meanValue, [x])

number 545.0018796992481
air_pressure_9am 918.9031798641051
air_temp_9am 65.02260949558733
avg_wind_direction_9am 142.30675564934037
avg_wind_speed_9am 5.48579305071369
max_wind_direction_9am 148.48042413321315
max_wind_speed_9am 6.999713658875691
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996037
relative_humidity_9am 34.07743985327709
relative_humidity_3pm 35.14838093290533
